# Simulating long experiments

This notebook introduces functionality for simulating experiments over hundreds or even thousands of cycles. 

In the interest of simplicity and running time, we consider a SPM with SEI effects leading to linear degradation.

- termination
- save_at_cycles
- summary variables
- starting solution

In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np

In [2]:
parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)
parameter_values.update(
    {
        "SEI kinetic rate constant [m.s-1]": 1e-15,
#         "SEI resistivity [Ohm.m]": 0,
    },
)
spm = pybamm.lithium_ion.SPM({"SEI": "ec reaction limited"})


In [3]:
esoh_model = pybamm.lithium_ion.ElectrodeSOH()
esoh_sim = pybamm.Simulation(esoh_model, parameter_values=parameter_values)
param = spm.param

Vmin = 3.0
Vmax = 4.2
Cn = parameter_values.evaluate(param.C_n_init)
Cp = parameter_values.evaluate(param.C_p_init)
n_Li_init = parameter_values.evaluate(param.n_Li_particles_init)
c_n_max = parameter_values.evaluate(param.c_n_max)
c_p_max = parameter_values.evaluate(param.c_p_max)

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init},
)
print(esoh_sol["x_100"].data[0])
print(esoh_sol["y_100"].data[0])

0.8334162315444245
0.03350230659015597


In [4]:
pybamm.set_logging_level("NOTICE")
parameter_values.update(
    {
        "Initial concentration in negative electrode [mol.m-3]": esoh_sol["x_100"].data[0] * c_n_max,
        "Initial concentration in positive electrode [mol.m-3]": esoh_sol["y_100"].data[0] * c_p_max,
    }
)
experiment = pybamm.Experiment([
    (f"Discharge at 1C until {Vmin}V",
     "Rest for 1 hour",
     f"Charge at 1C until {Vmax}V", 
     f"Hold at {Vmax}V until C/50"
    )
])
sim = pybamm.Simulation(spm, experiment=experiment, parameter_values=parameter_values, 
                        solver=pybamm.CasadiSolver("safe"))
spm_sol = sim.solve()

2021-05-05 13:08:55,035 - [NOTICE] simulation.solve(710): Cycle 1/1 (123.612 ms elapsed) --------------------
2021-05-05 13:08:55,036 - [NOTICE] simulation.solve(740): Cycle 1/1, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:08:55,182 - [NOTICE] simulation.solve(740): Cycle 1/1, step 2/4: Rest for 1 hour
2021-05-05 13:08:55,260 - [NOTICE] simulation.solve(740): Cycle 1/1, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:08:55,360 - [NOTICE] simulation.solve(740): Cycle 1/1, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:08:55,709 - [NOTICE] simulation.solve(835): Finish experiment simulation, took 797.584 ms


In [5]:
pybamm.set_logging_level("NOTICE")
experiment = pybamm.Experiment([
    (f"Discharge at 1C until {Vmin}V",
     "Rest for 1 hour",
    f"Charge at 1C until {Vmax}V", 
    f"Hold at {Vmax}V until C/50")
] * 500,
termination="80% capacity"
)
sim_100 = pybamm.Simulation(spm, experiment=experiment, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
spm_sol_100 = sim_100.solve()

2021-05-05 13:09:24,958 - [NOTICE] simulation.solve(710): Cycle 1/500 (34.238 ms elapsed) --------------------
2021-05-05 13:09:24,958 - [NOTICE] simulation.solve(740): Cycle 1/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:25,046 - [NOTICE] simulation.solve(740): Cycle 1/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:25,113 - [NOTICE] simulation.solve(740): Cycle 1/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:09:25,201 - [NOTICE] simulation.solve(740): Cycle 1/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:09:25,557 - [NOTICE] simulation.solve(818): Capacity is now 4.966 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:09:25,557 - [NOTICE] simulation.solve(710): Cycle 2/500 (633.923 ms elapsed) --------------------
2021-05-05 13:09:25,558 - [NOTICE] simulation.solve(740): Cycle 2/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:25,614 - [NOTICE] simulation.solve(740): Cycle 2/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:25,658 - [NO

2021-05-05 13:09:28,433 - [NOTICE] simulation.solve(740): Cycle 13/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:09:28,508 - [NOTICE] simulation.solve(818): Capacity is now 4.932 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:09:28,509 - [NOTICE] simulation.solve(710): Cycle 14/500 (3.586 s elapsed) --------------------
2021-05-05 13:09:28,510 - [NOTICE] simulation.solve(740): Cycle 14/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:28,564 - [NOTICE] simulation.solve(740): Cycle 14/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:28,602 - [NOTICE] simulation.solve(740): Cycle 14/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:09:28,654 - [NOTICE] simulation.solve(740): Cycle 14/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:09:28,729 - [NOTICE] simulation.solve(818): Capacity is now 4.929 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:09:28,730 - [NOTICE] simulation.solve(710): Cycle 15/500 (3.806 s elapsed) --------------------

2021-05-05 13:09:31,216 - [NOTICE] simulation.solve(740): Cycle 26/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:31,266 - [NOTICE] simulation.solve(740): Cycle 26/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:31,303 - [NOTICE] simulation.solve(740): Cycle 26/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:09:31,361 - [NOTICE] simulation.solve(740): Cycle 26/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:09:31,444 - [NOTICE] simulation.solve(818): Capacity is now 4.895 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:09:31,444 - [NOTICE] simulation.solve(710): Cycle 27/500 (6.521 s elapsed) --------------------
2021-05-05 13:09:31,445 - [NOTICE] simulation.solve(740): Cycle 27/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:31,496 - [NOTICE] simulation.solve(740): Cycle 27/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:31,534 - [NOTICE] simulation.solve(740): Cycle 27/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:09:31,581 - [NOT

2021-05-05 13:09:34,113 - [NOTICE] simulation.solve(740): Cycle 38/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:09:34,187 - [NOTICE] simulation.solve(818): Capacity is now 4.861 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:09:34,187 - [NOTICE] simulation.solve(710): Cycle 39/500 (9.264 s elapsed) --------------------
2021-05-05 13:09:34,188 - [NOTICE] simulation.solve(740): Cycle 39/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:34,239 - [NOTICE] simulation.solve(740): Cycle 39/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:34,276 - [NOTICE] simulation.solve(740): Cycle 39/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:09:34,323 - [NOTICE] simulation.solve(740): Cycle 39/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:09:34,398 - [NOTICE] simulation.solve(818): Capacity is now 4.859 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:09:34,398 - [NOTICE] simulation.solve(710): Cycle 40/500 (9.475 s elapsed) --------------------

2021-05-05 13:09:36,834 - [NOTICE] simulation.solve(740): Cycle 51/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:36,884 - [NOTICE] simulation.solve(740): Cycle 51/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:36,928 - [NOTICE] simulation.solve(740): Cycle 51/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:09:36,977 - [NOTICE] simulation.solve(740): Cycle 51/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:09:37,174 - [NOTICE] simulation.solve(818): Capacity is now 4.825 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:09:37,175 - [NOTICE] simulation.solve(710): Cycle 52/500 (12.251 s elapsed) --------------------
2021-05-05 13:09:37,175 - [NOTICE] simulation.solve(740): Cycle 52/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:09:37,224 - [NOTICE] simulation.solve(740): Cycle 52/500, step 2/4: Rest for 1 hour
2021-05-05 13:09:37,266 - [NOTICE] simulation.solve(740): Cycle 52/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:09:37,311 - [NO

...

2021-05-05 13:10:08,320 - [NOTICE] simulation.solve(740): Cycle 188/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:10:08,400 - [NOTICE] simulation.solve(818): Capacity is now 4.467 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:10:08,401 - [NOTICE] simulation.solve(710): Cycle 189/500 (43.478 s elapsed) --------------------
2021-05-05 13:10:08,402 - [NOTICE] simulation.solve(740): Cycle 189/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:10:08,451 - [NOTICE] simulation.solve(740): Cycle 189/500, step 2/4: Rest for 1 hour
2021-05-05 13:10:08,490 - [NOTICE] simulation.solve(740): Cycle 189/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:10:08,535 - [NOTICE] simulation.solve(740): Cycle 189/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:10:08,615 - [NOTICE] simulation.solve(818): Capacity is now 4.465 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:10:08,615 - [NOTICE] simulation.solve(710): Cycle 190/500 (43.692 s elapsed) -----------

2021-05-05 13:10:11,046 - [NOTICE] simulation.solve(740): Cycle 201/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:10:11,095 - [NOTICE] simulation.solve(740): Cycle 201/500, step 2/4: Rest for 1 hour
2021-05-05 13:10:11,135 - [NOTICE] simulation.solve(740): Cycle 201/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:10:11,181 - [NOTICE] simulation.solve(740): Cycle 201/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:10:11,267 - [NOTICE] simulation.solve(818): Capacity is now 4.438 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:10:11,268 - [NOTICE] simulation.solve(710): Cycle 202/500 (46.344 s elapsed) --------------------
2021-05-05 13:10:11,268 - [NOTICE] simulation.solve(740): Cycle 202/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:10:11,319 - [NOTICE] simulation.solve(740): Cycle 202/500, step 2/4: Rest for 1 hour
2021-05-05 13:10:11,357 - [NOTICE] simulation.solve(740): Cycle 202/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:10:11,4

2021-05-05 13:10:13,892 - [NOTICE] simulation.solve(740): Cycle 213/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:10:13,993 - [NOTICE] simulation.solve(818): Capacity is now 4.408 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:10:13,994 - [NOTICE] simulation.solve(710): Cycle 214/500 (49.070 s elapsed) --------------------
2021-05-05 13:10:13,994 - [NOTICE] simulation.solve(740): Cycle 214/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:10:14,048 - [NOTICE] simulation.solve(740): Cycle 214/500, step 2/4: Rest for 1 hour
2021-05-05 13:10:14,091 - [NOTICE] simulation.solve(740): Cycle 214/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:10:14,135 - [NOTICE] simulation.solve(740): Cycle 214/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:10:14,219 - [NOTICE] simulation.solve(818): Capacity is now 4.406 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:10:14,220 - [NOTICE] simulation.solve(710): Cycle 215/500 (49.296 s elapsed) -----------

2021-05-05 13:10:16,693 - [NOTICE] simulation.solve(740): Cycle 226/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:10:16,739 - [NOTICE] simulation.solve(740): Cycle 226/500, step 2/4: Rest for 1 hour
2021-05-05 13:10:16,776 - [NOTICE] simulation.solve(740): Cycle 226/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:10:16,823 - [NOTICE] simulation.solve(740): Cycle 226/500, step 4/4: Hold at 4.2V until C/50
2021-05-05 13:10:16,911 - [NOTICE] simulation.solve(818): Capacity is now 4.377 Ah (originally 4.966 Ah, will stop at 3.973 Ah)
2021-05-05 13:10:16,912 - [NOTICE] simulation.solve(710): Cycle 227/500 (51.988 s elapsed) --------------------
2021-05-05 13:10:16,912 - [NOTICE] simulation.solve(740): Cycle 227/500, step 1/4: Discharge at 1C until 3.0V
2021-05-05 13:10:16,962 - [NOTICE] simulation.solve(740): Cycle 227/500, step 2/4: Rest for 1 hour
2021-05-05 13:10:17,000 - [NOTICE] simulation.solve(740): Cycle 227/500, step 3/4: Charge at 1C until 4.2V
2021-05-05 13:10:17,0

SolverError: Could not find acceptable solution: solver terminated successfully, but maximum solution error (0.0379440280770158) above tolerance (1e-06)